In [16]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *
import re
from sklearn.cluster import KMeans

In [2]:
# df = pd.read_csv("data/genome-scores.csv")
# df2 = pd.read_csv("data/genome-tags.csv")
# df3 = pd.read_csv("data/links.csv")
# df4 = pd.read_csv("data/movies.csv")
df5 = pd.read_csv("ratings.csv")
genreMatrix = pd.read_csv('clean_data/genreMatrix.csv')
links = pd.read_csv('links.csv')
other_info = pd.read_csv('clean_data/imdb_details_clean.csv')

In [3]:
ratings = df5.groupby('movieId').mean().drop(columns = ['userId', 'timestamp'])

In [4]:
ids = links.drop(columns=['imdbId', 'tmdbId']).rename(columns={'imdbID_formatted':'imdbID'})

In [5]:
movies_other = other_info.drop(columns=['Rated','Genre','Director','Writer','Actors','Type','DVD','BoxOffice','Production','Website','Season','Episode','totalSeasons','seriesID','Awards','Country','Language','Metascore','imdbRating'])

In [6]:
movies = pd.merge(ids, movies_other,how='inner',on='imdbID')

In [7]:
movies_ratings = pd.merge(movies, ratings, how='inner', on='movieId')

In [8]:
movies_genres_ratings = pd.merge(movies_ratings, genreMatrix, how='inner', on='imdbID')

In [9]:
other_info.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Season', 'Episode',
       'totalSeasons', 'seriesID'],
      dtype='object')

In [10]:
votes = []
temp = []
for i in range(0, len(other_info)):
    text = float(str(other_info['imdbVotes'][i]).replace(',',''))/2177949
    movieid = other_info['imdbID'][i]
#     year = float(str(other_info['Year'][i]).replace('Äì',''))
    mscore = other_info['Metascore'][i]
    temp = [text,movieid]
    votes.append(temp)

In [11]:
votes_score = pd.DataFrame(votes, columns =['votes','imdbID'])

In [12]:
more_info = pd.merge(movies_genres_ratings,votes_score,how='inner',on='imdbID')

In [13]:
more_info.to_csv('final_movies.csv')

In [14]:
more_info.columns

Index(['movieId', 'imdbID', 'Title', 'Year', 'Released', 'Runtime', 'Plot',
       'Poster', 'Ratings', 'imdbVotes', 'rating', 'Action', 'Adult',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
       'Western', 'votes'],
      dtype='object')

In [17]:
full = pd.read_csv('final_movies.csv').dropna()
selections = full.drop(columns=['Unnamed: 0','movieId','imdbID','Title','Year','Ratings','Released','Runtime','Plot','Poster','imdbVotes'])

kmeans = KMeans(n_clusters=800, random_state = 42)
kmeans.fit(selections)
clusters = kmeans.predict(selections)
full['clusters']=clusters

full.to_csv('clustered_movies.csv')